In [1]:
#importing the pandas library
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
#reading the data set 
df = pd.read_csv("/kaggle/input/armut-arl/armut_data.csv")
df.head() #Let's get the first 5 observations

,UserId,ServiceId,CategoryId,CreateDate
0,25446,4,5,2017-08-06 16:11:00
1,22948,48,5,2017-08-06 16:12:00
2,10618,0,8,2017-08-06 16:13:00
3,7256,9,4,2017-08-06 16:14:00
4,25446,48,5,2017-08-06 16:16:00


In [3]:
df.info()#getting an overview about dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162523 entries, 0 to 162522
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   UserId      162523 non-null  int64 
 1   ServiceId   162523 non-null  int64 
 2   CategoryId  162523 non-null  int64 
 3   CreateDate  162523 non-null  object
dtypes: int64(3), object(1)
memory usage: 5.0+ MB


In [4]:
#Since we want to see together which service you received from the category, we combine the two
df["hizmet"] = df["ServiceId"].astype(str) + "_" + df["CategoryId"].astype(str)
df.head()

,UserId,ServiceId,CategoryId,CreateDate,hizmet
0,25446,4,5,2017-08-06 16:11:00,4_5
1,22948,48,5,2017-08-06 16:12:00,48_5
2,10618,0,8,2017-08-06 16:13:00,0_8
3,7256,9,4,2017-08-06 16:14:00,9_4
4,25446,48,5,2017-08-06 16:16:00,48_5


In [5]:
#conversion to date data structure 
df["CreateDate"] = pd.to_datetime(df["CreateDate"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162523 entries, 0 to 162522
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   UserId      162523 non-null  int64         
 1   ServiceId   162523 non-null  int64         
 2   CategoryId  162523 non-null  int64         
 3   CreateDate  162523 non-null  datetime64[ns]
 4   hizmet      162523 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 6.2+ MB


In [6]:
#Let's create a new variable using the date, month and year the service was used 
df["New_Date"] = df["CreateDate"].dt.to_period('M')
df.head()

,UserId,ServiceId,CategoryId,CreateDate,hizmet,New_Date
0,25446,4,5,2017-08-06 16:11:00,4_5,2017-08
1,22948,48,5,2017-08-06 16:12:00,48_5,2017-08
2,10618,0,8,2017-08-06 16:13:00,0_8,2017-08
3,7256,9,4,2017-08-06 16:14:00,9_4,2017-08
4,25446,48,5,2017-08-06 16:16:00,48_5,2017-08


In [7]:
#We use userid with the variable we just created to create SepetID
df["SepetID"] = df["UserId"].astype(str) + "_" + df["New_Date"].astype(str)
df.head()

,UserId,ServiceId,CategoryId,CreateDate,hizmet,New_Date,SepetID
0,25446,4,5,2017-08-06 16:11:00,4_5,2017-08,25446_2017-08
1,22948,48,5,2017-08-06 16:12:00,48_5,2017-08,22948_2017-08
2,10618,0,8,2017-08-06 16:13:00,0_8,2017-08,10618_2017-08
3,7256,9,4,2017-08-06 16:14:00,9_4,2017-08,7256_2017-08
4,25446,48,5,2017-08-06 16:16:00,48_5,2017-08,25446_2017-08


In [8]:
# let's create a new dataframe to calculate association rules.
# if there is an empty value, fill it with zero, then go through all the observations and print the number one for those that are greater than zero and zero for those that are not.
# We have converted the table to binary values 
df_new = df.pivot_table("ServiceId","SepetID", "hizmet").fillna(0).applymap(lambda x:1 if x > 0 else 0)

/tmp/ipykernel_18/1319952147.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_new = df.pivot_table("ServiceId","SepetID", "hizmet").fillna(0).applymap(lambda x:1 if x > 0 else 0)


In [9]:
#Let's write a function to calculate association rules
def create_rules(dataframe):
    frequent_itemsets = apriori(dataframe, min_support=0.01, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
    return rules

rules = create_rules(df_new)
rules

/opt/conda/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(2_0),(13_11),0.130286,0.056627,0.012819,0.098394,1.737574,0.005442,1.046325,0.488074
1,(13_11),(2_0),0.056627,0.130286,0.012819,0.226382,1.737574,0.005442,1.124216,0.449965
2,(2_0),(15_1),0.130286,0.120963,0.033951,0.260588,2.154278,0.018191,1.188833,0.616073
3,(15_1),(2_0),0.120963,0.130286,0.033951,0.280673,2.154278,0.018191,1.209066,0.609539
4,(33_4),(15_1),0.027310,0.120963,0.011233,0.411311,3.400299,0.007929,1.493211,0.725728
5,(15_1),(33_4),0.120963,0.027310,0.011233,0.092861,3.400299,0.007929,1.072262,0.803047
6,(38_4),(15_1),0.066568,0.120963,0.011177,0.167897,1.388001,0.003124,1.056404,0.299475
7,(15_1),(38_4),0.120963,0.066568,0.011177,0.092397,1.388001,0.003124,1.028458,0.318007
8,(49_1),(15_1),0.067762,0.120963,0.010011,0.147741,1.221375,0.001815,1.031420,0.194425
9,(15_1),(49_1),0.120963,0.067762,0.010011,0.082763,1.221375,0.001815,1.016354,0.206192


In [10]:
#creates the services we will offer to the user according to the lift variable.
def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])
    return recommendation_list[0:rec_count]

arl_recommender(rules, "2_0" )

['22_0']